<a href="https://colab.research.google.com/github/MarinaSupiot/projet_7/blob/main/MLFLOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import gc
from hyperopt import hp
from hyperopt import tpe
from hyperopt import fmin
from hyperopt import space_eval, STATUS_OK
import plotly.express as px
from plotly.offline import iplot

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

import lightgbm as lgb
#!pip install lime
#from lime import lime_tabular

import matplotlib as mpl
%matplotlib inline
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import pickle
import random
import re
import seaborn as sns
!pip install shap
import shap


import sklearn
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, fbeta_score

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

!pip install scikit-plot
!pip install kds

import scikitplot as skplt
import kds

import pickle
import os



import time

# Versions
print('Version des librairies utilisées :')


print('Matplotlib            : ' + mpl.__version__)
print('Missingno             : ' + msno.__version__)
print('NumPy                 : ' + np.version.full_version)
print('Pandas                : ' + pd.__version__)
print('Seaborn               : ' + sns.__version__)
print('Shap                  : ' + shap.__version__)
print('Sklearn               : ' + sklearn.__version__ )

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.5/533.5 kB 7.7 MB/s eta 0:00:00
Version des librairies utilisées :
Matplotlib            : 3.7.1
Missingno             : 0.5.2
NumPy                 : 1.23.5
Pandas                : 1.5.3
Seaborn               : 0.12.2
Shap                  : 0.44.0
Sklearn               : 1.2.2


In [3]:
def cout_metier(y_true, y_pred, fn_value=-10, fp_value=-1, vp_value=0, vn_value=1):
    mat_conf = confusion_matrix(y_true, y_pred)
    vn = mat_conf[0, 0]
    fn = mat_conf[1, 0]
    fp = mat_conf[0, 1]
    vp = mat_conf[1, 1]
    J = vp * vp_value + vn * vn_value + fp * fp_value + fn * fn_value
    # Coût maximum
    max_J = (fp + vn)*vn_value + (fn + vp)*vp_value

    # Coût minimum
    min_J = (fp + vn)*fp_value + (fn + vp)*fn_value

    # Coût normalisé entre 0 et 1
    J_normalized = (J - min_J)/(max_J - min_J)

    return J_normalized

Connexion MLFLOW

In [4]:
!pip install mlflow
import mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.0 MB/s eta 0:00:00


In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [5]:
seed = 42

In [6]:
#chargement des train set et val set
train_set = pickle.load( open( "/content/drive/MyDrive/Projet 7/train_set.p", "rb" ) )
val_set = pickle.load( open( "/content/drive/MyDrive/Projet 7/val_set.p", "rb" ) )
train_labels = pickle.load( open( "/content/drive/MyDrive/Projet 7/train_labels.p", "rb" ) )
val_labels = pickle.load( open( "/content/drive/MyDrive/Projet 7/val_labels.p", "rb" ) )


In [7]:
train_sample_set = pickle.load( open( "/content/drive/MyDrive/Projet 7/train_sample_set.p", "rb" ) )
val_sample_set = pickle.load( open( "/content/drive/MyDrive/Projet 7/val_sample_set.p", "rb" ) )
train_sample_labels = pickle.load( open( "/content/drive/MyDrive/Projet 7/train_sample_labels.p", "rb" ) )
val_sample_labels = pickle.load( open( "/content/drive/MyDrive/Projet 7/val_sample_labels.p", "rb" ) )

# AdaBoost

In [11]:
# Définition d'une expérience (Une expérience peut contenir plusieurs *runs*)
mlflow.set_experiment("AdaBoost")
mlflow.sklearn.autolog()  # Ici on fait appel à l'API MLflow qui intègre les algos de Scikit-Learn.
                          # C'est grâce à ce code qu'on enregistre toutes les métriques et métadonnées de cette expérience (Module MLflow Tracking)

with open('/content/drive/MyDrive/Projet 7/Adaboost/model_smote.pkl', 'rb') as file:
    model = pickle.load(file)
with mlflow.start_run(run_name='AdaBoost_Smote'):
    #model = build_pipeline(algo_ml=model)
    #model.fit(train_sample_set.drop(columns=['SK_ID_CURR']), train_sample_labels)

    # Prédictions sur les données d'entraînement
    #train_preds = model.predict(train_sample_set.drop(columns=['SK_ID_CURR']))
    # Prédictions sur les données de validation
    val_preds = model.predict(val_sample_set.drop(columns=['TARGET']))

    # Calcul des métriques
    roc_auc = roc_auc_score(val_sample_labels, val_preds)
    recall = recall_score(val_sample_labels, val_preds)
    precision = precision_score(val_sample_labels, val_preds)
    f1 = f1_score(val_sample_labels, val_preds)
    cout = cout_metier(val_sample_labels, val_preds)


    # Enregistrement des métriques dans les logs MLflow
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("cout", cout)

    print(roc_auc)
    print(recall)
    print(precision)
    print(f1)
    print(cout)

mlflow.end_run()

2023/12/07 13:22:15 INFO mlflow.tracking.fluent: Experiment with name 'AdaBoost' does not exist. Creating a new experiment.


0.5160987155312156
0.03825136612021858
0.3888888888888889
0.06965174129353234
0.6738653001464129


In [12]:
mlflow.set_experiment("AdaBoost")
mlflow.sklearn.autolog()  # Ici on fait appel à l'API MLflow qui intègre les algos de Scikit-Learn.
                          # C'est grâce à ce code qu'on enregistre toutes les métriques et métadonnées de cette expérience (Module MLflow Tracking)

with open('/content/drive/MyDrive/Projet 7/Adaboost/model_0.pkl', 'rb') as file:
    model = pickle.load(file)
with mlflow.start_run(run_name='AdaBoost_wight'):
    #model = build_pipeline(algo_ml=model)
    #model.fit(train_sample_set.drop(columns=['SK_ID_CURR']), train_sample_labels)

    # Prédictions sur les données d'entraînement
    #train_preds = model.predict(train_sample_set.drop(columns=['SK_ID_CURR']))
    # Prédictions sur les données de validation
    val_preds = model.predict(val_sample_set.drop(columns=['TARGET']))

    # Calcul des métriques
    roc_auc = roc_auc_score(val_sample_labels, val_preds)
    recall = recall_score(val_sample_labels, val_preds)
    precision = precision_score(val_sample_labels, val_preds)
    f1 = f1_score(val_sample_labels, val_preds)
    cout = cout_metier(val_sample_labels, val_preds)


    # Enregistrement des métriques dans les logs MLflow
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("cout", cout)

    print(roc_auc)
    print(recall)
    print(precision)
    print(f1)
    print(cout)

mlflow.end_run()

0.5
0.0
0.0
0.0
0.6650805270863837


Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.


In [13]:
mlflow.set_experiment("AdaBoost")
mlflow.sklearn.autolog()  # Ici on fait appel à l'API MLflow qui intègre les algos de Scikit-Learn.
                          # C'est grâce à ce code qu'on enregistre toutes les métriques et métadonnées de cette expérience (Module MLflow Tracking)

with open('/content/drive/MyDrive/Projet 7/Adaboost/model_us.pkl', 'rb') as file:
    model = pickle.load(file)
with mlflow.start_run(run_name='AdaBoost_Undersampling'):
    #model = build_pipeline(algo_ml=model)
    #model.fit(train_sample_set.drop(columns=['SK_ID_CURR']), train_sample_labels)

    # Prédictions sur les données d'entraînement
    #train_preds = model.predict(train_sample_set.drop(columns=['SK_ID_CURR']))
    # Prédictions sur les données de validation
    val_preds = model.predict(val_sample_set.drop(columns=['TARGET']))

    # Calcul des métriques
    roc_auc = roc_auc_score(val_sample_labels, val_preds)
    recall = recall_score(val_sample_labels, val_preds)
    precision = precision_score(val_sample_labels, val_preds)
    f1 = f1_score(val_sample_labels, val_preds)
    cout = cout_metier(val_sample_labels, val_preds)


    # Enregistrement des métriques dans les logs MLflow
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("cout", cout)

    print(roc_auc)
    print(recall)
    print(precision)
    print(f1)
    print(cout)

mlflow.end_run()

0.5013563461058431
0.00546448087431694
0.16666666666666666
0.010582010582010581
0.6650805270863837


In [14]:
mlflow.set_experiment("AdaBoost")
mlflow.sklearn.autolog()  # Ici on fait appel à l'API MLflow qui intègre les algos de Scikit-Learn.
                          # C'est grâce à ce code qu'on enregistre toutes les métriques et métadonnées de cette expérience (Module MLflow Tracking)

with open('/content/drive/MyDrive/Projet 7/Adaboost/model_su02.pkl', 'rb') as file:
    model = pickle.load(file)
with mlflow.start_run(run_name='AdaBoost_SMOTE_02'):
    #model = build_pipeline(algo_ml=model)
    #model.fit(train_sample_set.drop(columns=['SK_ID_CURR']), train_sample_labels)

    # Prédictions sur les données d'entraînement
    #train_preds = model.predict(train_sample_set.drop(columns=['SK_ID_CURR']))
    # Prédictions sur les données de validation
    val_preds = model.predict(val_sample_set.drop(columns=['TARGET']))

    # Calcul des métriques
    roc_auc = roc_auc_score(val_sample_labels, val_preds)
    recall = recall_score(val_sample_labels, val_preds)
    precision = precision_score(val_sample_labels, val_preds)
    f1 = f1_score(val_sample_labels, val_preds)
    cout = cout_metier(val_sample_labels, val_preds)


    # Enregistrement des métriques dans les logs MLflow
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("cout", cout)

    print(roc_auc)
    print(recall)
    print(precision)
    print(f1)
    print(cout)

mlflow.end_run()

0.5
0.0
0.0
0.0
0.6650805270863837


Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.


In [15]:
mlflow.set_experiment("AdaBoost")
mlflow.sklearn.autolog()  # Ici on fait appel à l'API MLflow qui intègre les algos de Scikit-Learn.
                          # C'est grâce à ce code qu'on enregistre toutes les métriques et métadonnées de cette expérience (Module MLflow Tracking)

with open('/content/drive/MyDrive/Projet 7/Adaboost/model_su04.pkl', 'rb') as file:
    model = pickle.load(file)
with mlflow.start_run(run_name='AdaBoost_SMOTE_04'):
    #model = build_pipeline(algo_ml=model)
    #model.fit(train_sample_set.drop(columns=['SK_ID_CURR']), train_sample_labels)

    # Prédictions sur les données d'entraînement
    #train_preds = model.predict(train_sample_set.drop(columns=['SK_ID_CURR']))
    # Prédictions sur les données de validation
    val_preds = model.predict(val_sample_set.drop(columns=['TARGET']))

    # Calcul des métriques
    roc_auc = roc_auc_score(val_sample_labels, val_preds)
    recall = recall_score(val_sample_labels, val_preds)
    precision = precision_score(val_sample_labels, val_preds)
    f1 = f1_score(val_sample_labels, val_preds)
    cout = cout_metier(val_sample_labels, val_preds)


    # Enregistrement des métriques dans les logs MLflow
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("cout", cout)

    print(roc_auc)
    print(recall)
    print(precision)
    print(f1)
    print(cout)

mlflow.end_run()

0.5
0.0
0.0
0.0
0.6650805270863837


Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.


# Dummy Regressor

In [18]:
mlflow.set_experiment("Dummy")
mlflow.sklearn.autolog()  # Ici on fait appel à l'API MLflow qui intègre les algos de Scikit-Learn.
                          # C'est grâce à ce code qu'on enregistre toutes les métriques et métadonnées de cette expérience (Module MLflow Tracking)

with open('/content/drive/MyDrive/Projet 7/DummyRegressor/model_0.pkl', 'rb') as file:
    model = pickle.load(file)
with mlflow.start_run(run_name='Dummy_poids'):
    #model = build_pipeline(algo_ml=model)
    #model.fit(train_sample_set.drop(columns=['SK_ID_CURR']), train_sample_labels)

    # Prédictions sur les données d'entraînement
    #train_preds = model.predict(train_sample_set.drop(columns=['SK_ID_CURR']))
    # Prédictions sur les données de validation
    val_preds = model.predict(val_sample_set.drop(columns=['TARGET']))

    # Calcul des métriques
    roc_auc = roc_auc_score(val_sample_labels, val_preds)
    recall = recall_score(val_sample_labels, val_preds)
    precision = precision_score(val_sample_labels, val_preds)
    f1 = f1_score(val_sample_labels, val_preds)
    cout = cout_metier(val_sample_labels, val_preds)


    # Enregistrement des métriques dans les logs MLflow
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("cout", cout)

    print(roc_auc)
    print(recall)
    print(precision)
    print(f1)
    print(cout)

mlflow.end_run()

2023/12/07 13:25:35 INFO mlflow.tracking.fluent: Experiment with name 'Dummy' does not exist. Creating a new experiment.


0.49094014934844266
0.07103825136612021
0.07428571428571429
0.07262569832402235
0.6295754026354319


In [21]:
mlflow.set_experiment("Dummy")
mlflow.sklearn.autolog()  # Ici on fait appel à l'API MLflow qui intègre les algos de Scikit-Learn.
                          # C'est grâce à ce code qu'on enregistre toutes les métriques et métadonnées de cette expérience (Module MLflow Tracking)

with open('/content/drive/MyDrive/Projet 7/DummyRegressor/model_smote.pkl', 'rb') as file:
    model = pickle.load(file)
with mlflow.start_run(run_name='SMOTE'):
    #model = build_pipeline(algo_ml=model)
    #model.fit(train_sample_set.drop(columns=['SK_ID_CURR']), train_sample_labels)

    # Prédictions sur les données d'entraînement
    #train_preds = model.predict(train_sample_set.drop(columns=['SK_ID_CURR']))
    # Prédictions sur les données de validation
    val_preds = model.predict(val_sample_set.drop(columns=['TARGET']))

    # Calcul des métriques
    roc_auc = roc_auc_score(val_sample_labels, val_preds)
    recall = recall_score(val_sample_labels, val_preds)
    precision = precision_score(val_sample_labels, val_preds)
    f1 = f1_score(val_sample_labels, val_preds)
    cout = cout_metier(val_sample_labels, val_preds)


    # Enregistrement des métriques dans les logs MLflow
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("cout", cout)

    print(roc_auc)
    print(recall)
    print(precision)
    print(f1)
    print(cout)

mlflow.end_run()

0.4990661963062877
0.4918032786885246
0.0911854103343465
0.15384615384615385
0.5014641288433382


## Undersampling

In [24]:
mlflow.set_experiment("Dummy")
mlflow.sklearn.autolog()  # Ici on fait appel à l'API MLflow qui intègre les algos de Scikit-Learn.
                          # C'est grâce à ce code qu'on enregistre toutes les métriques et métadonnées de cette expérience (Module MLflow Tracking)

with open('/content/drive/MyDrive/Projet 7/DummyRegressor/model_us.pkl', 'rb') as file:
    model = pickle.load(file)
with mlflow.start_run(run_name='Undersampling'):
    #model = build_pipeline(algo_ml=model)
    #model.fit(train_sample_set.drop(columns=['SK_ID_CURR']), train_sample_labels)

    # Prédictions sur les données d'entraînement
    #train_preds = model.predict(train_sample_set.drop(columns=['SK_ID_CURR']))
    # Prédictions sur les données de validation
    val_preds = model.predict(val_sample_set.drop(columns=['TARGET']))

    # Calcul des métriques
    roc_auc = roc_auc_score(val_sample_labels, val_preds)
    recall = recall_score(val_sample_labels, val_preds)
    precision = precision_score(val_sample_labels, val_preds)
    f1 = f1_score(val_sample_labels, val_preds)
    cout = cout_metier(val_sample_labels, val_preds)


    # Enregistrement des métriques dans les logs MLflow
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("cout", cout)

    print(roc_auc)
    print(recall)
    print(precision)
    print(f1)
    print(cout)

mlflow.end_run()

0.4990661963062877
0.4918032786885246
0.0911854103343465
0.15384615384615385
0.5014641288433382


In [25]:
mlflow.set_experiment("Dummy")
mlflow.sklearn.autolog()  # Ici on fait appel à l'API MLflow qui intègre les algos de Scikit-Learn.
                          # C'est grâce à ce code qu'on enregistre toutes les métriques et métadonnées de cette expérience (Module MLflow Tracking)

with open('/content/drive/MyDrive/Projet 7/DummyRegressor/model_su04.pkl', 'rb') as file:
    model = pickle.load(file)
with mlflow.start_run(run_name='Smote 0.4'):
    #model = build_pipeline(algo_ml=model)
    #model.fit(train_sample_set.drop(columns=['SK_ID_CURR']), train_sample_labels)

    # Prédictions sur les données d'entraînement
    #train_preds = model.predict(train_sample_set.drop(columns=['SK_ID_CURR']))
    # Prédictions sur les données de validation
    val_preds = model.predict(val_sample_set.drop(columns=['TARGET']))

    # Calcul des métriques
    roc_auc = roc_auc_score(val_sample_labels, val_preds)
    recall = recall_score(val_sample_labels, val_preds)
    precision = precision_score(val_sample_labels, val_preds)
    f1 = f1_score(val_sample_labels, val_preds)
    cout = cout_metier(val_sample_labels, val_preds)


    # Enregistrement des métriques dans les logs MLflow
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("cout", cout)

    print(roc_auc)
    print(recall)
    print(precision)
    print(f1)
    print(cout)

mlflow.end_run()

0.4990661963062877
0.4918032786885246
0.0911854103343465
0.15384615384615385
0.5014641288433382


# Light GBM

In [28]:
mlflow.set_experiment("Light GBM")
mlflow.sklearn.autolog()  # Ici on fait appel à l'API MLflow qui intègre les algos de Scikit-Learn.
                          # C'est grâce à ce code qu'on enregistre toutes les métriques et métadonnées de cette expérience (Module MLflow Tracking)

with open('/content/drive/MyDrive/Projet 7/model_0.pkl', 'rb') as file:
    model = pickle.load(file)
with mlflow.start_run(run_name='LightGBM_poids'):
    #model = build_pipeline(algo_ml=model)
    #model.fit(train_sample_set.drop(columns=['SK_ID_CURR']), train_sample_labels)

    # Prédictions sur les données d'entraînement
    #train_preds = model.predict(train_sample_set.drop(columns=['SK_ID_CURR']))
    # Prédictions sur les données de validation
    val_preds = model.predict(val_sample_set.drop(columns=['TARGET']))

    # Calcul des métriques
    roc_auc = roc_auc_score(val_sample_labels, val_preds)
    recall = recall_score(val_sample_labels, val_preds)
    precision = precision_score(val_sample_labels, val_preds)
    f1 = f1_score(val_sample_labels, val_preds)
    cout = cout_metier(val_sample_labels, val_preds)


    # Enregistrement des métriques dans les logs MLflow
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("cout", cout)

    print(roc_auc)
    print(recall)
    print(precision)
    print(f1)
    print(cout)

mlflow.end_run()

2023/12/07 13:33:05 INFO mlflow.tracking.fluent: Experiment with name 'Light GBM' does not exist. Creating a new experiment.


0.6541888839767708
0.5136612021857924
0.2012847965738758
0.28923076923076924
0.7005856515373353


In [32]:
mlflow.set_experiment("Light GBM")
mlflow.sklearn.autolog()  # Ici on fait appel à l'API MLflow qui intègre les algos de Scikit-Learn.
                          # C'est grâce à ce code qu'on enregistre toutes les métriques et métadonnées de cette expérience (Module MLflow Tracking)

with open('/content/drive/MyDrive/Projet 7/model_smote.pkl', 'rb') as file:
    model = pickle.load(file)
with mlflow.start_run(run_name='SMOTE'):
    #model = build_pipeline(algo_ml=model)
    #model.fit(train_sample_set.drop(columns=['SK_ID_CURR']), train_sample_labels)

    # Prédictions sur les données d'entraînement
    #train_preds = model.predict(train_sample_set.drop(columns=['SK_ID_CURR']))
    # Prédictions sur les données de validation
    val_preds = model.predict(val_sample_set.drop(columns=['TARGET']))

    # Calcul des métriques
    roc_auc = roc_auc_score(val_sample_labels, val_preds)
    recall = recall_score(val_sample_labels, val_preds)
    precision = precision_score(val_sample_labels, val_preds)
    f1 = f1_score(val_sample_labels, val_preds)
    cout = cout_metier(val_sample_labels, val_preds)


    # Enregistrement des métriques dans les logs MLflow
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("cout", cout)

    print(roc_auc)
    print(recall)
    print(precision)
    print(f1)
    print(cout)

mlflow.end_run()

0.6633494831749928
0.5683060109289617
0.19152854511970535
0.2865013774104683
0.6947291361639825


In [36]:
mlflow.set_experiment("Light GBM")
mlflow.sklearn.autolog()  # Ici on fait appel à l'API MLflow qui intègre les algos de Scikit-Learn.
                          # C'est grâce à ce code qu'on enregistre toutes les métriques et métadonnées de cette expérience (Module MLflow Tracking)

with open('/content/drive/MyDrive/Projet 7/model_us.pkl', 'rb') as file:
    model = pickle.load(file)
with mlflow.start_run(run_name='Undersampling'):
    #model = build_pipeline(algo_ml=model)
    #model.fit(train_sample_set.drop(columns=['SK_ID_CURR']), train_sample_labels)

    # Prédictions sur les données d'entraînement
    #train_preds = model.predict(train_sample_set.drop(columns=['SK_ID_CURR']))
    # Prédictions sur les données de validation
    val_preds = model.predict(val_sample_set.drop(columns=['TARGET']))

    # Calcul des métriques
    roc_auc = roc_auc_score(val_sample_labels, val_preds)
    recall = recall_score(val_sample_labels, val_preds)
    precision = precision_score(val_sample_labels, val_preds)
    f1 = f1_score(val_sample_labels, val_preds)
    cout = cout_metier(val_sample_labels, val_preds)


    # Enregistrement des métriques dans les logs MLflow
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("cout", cout)

    print(roc_auc)
    print(recall)
    print(precision)
    print(f1)
    print(cout)

mlflow.end_run()

0.6313264824321602
0.366120218579235
0.2627450980392157
0.3059360730593607
0.718887262079063


In [39]:
mlflow.set_experiment("Light GBM")
mlflow.sklearn.autolog()  # Ici on fait appel à l'API MLflow qui intègre les algos de Scikit-Learn.
                          # C'est grâce à ce code qu'on enregistre toutes les métriques et métadonnées de cette expérience (Module MLflow Tracking)

with open('/content/drive/MyDrive/Projet 7/model_su04.pkl', 'rb') as file:
    model = pickle.load(file)
with mlflow.start_run(run_name='SMOTE 0.4'):
    #model = build_pipeline(algo_ml=model)
    #model.fit(train_sample_set.drop(columns=['SK_ID_CURR']), train_sample_labels)

    # Prédictions sur les données d'entraînement
    #train_preds = model.predict(train_sample_set.drop(columns=['SK_ID_CURR']))
    # Prédictions sur les données de validation
    val_preds = model.predict(val_sample_set.drop(columns=['TARGET']))

    # Calcul des métriques
    roc_auc = roc_auc_score(val_sample_labels, val_preds)
    recall = recall_score(val_sample_labels, val_preds)
    precision = precision_score(val_sample_labels, val_preds)
    f1 = f1_score(val_sample_labels, val_preds)
    cout = cout_metier(val_sample_labels, val_preds)


    # Enregistrement des métriques dans les logs MLflow
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("cout", cout)

    print(roc_auc)
    print(recall)
    print(precision)
    print(f1)
    print(cout)

mlflow.end_run()

0.6570594657018866
0.5573770491803278
0.1875
0.28060522696011003
0.6899707174231332


In [8]:
!pip install pyngrok

In [40]:

from pyngrok import ngrok
# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2YGm2G54kBEMvnAj43IFcf4QK1O_7kd3J9UnbA4TGcvw9W5Np"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://aa58-34-90-174-74.ngrok-free.app


In [41]:
!mlflow ui

[2023-12-07 13:37:50 +0000] [7917] [INFO] Starting gunicorn 21.2.0
[2023-12-07 13:37:50 +0000] [7917] [INFO] Listening at: http://127.0.0.1:5000 (7917)
[2023-12-07 13:37:50 +0000] [7917] [INFO] Using worker: sync
[2023-12-07 13:37:50 +0000] [7918] [INFO] Booting worker with pid: 7918
[2023-12-07 13:37:50 +0000] [7919] [INFO] Booting worker with pid: 7919
[2023-12-07 13:37:50 +0000] [7920] [INFO] Booting worker with pid: 7920
[2023-12-07 13:37:50 +0000] [7921] [INFO] Booting worker with pid: 7921

[2023-12-07 13:38:27 +0000] [7917] [INFO] Handling signal: int
Aborted!
[2023-12-07 13:38:27 +0000] [7920] [INFO] Worker exiting (pid: 7920)
[2023-12-07 13:38:27 +0000] [7919] [INFO] Worker exiting (pid: 7919)
[2023-12-07 13:38:27 +0000] [7921] [INFO] Worker exiting (pid: 7921)
[2023-12-07 13:38:27 +0000] [7918] [INFO] Worker exiting (pid: 7918)
[2023-12-07 13:38:28 +0000] [7917] [INFO] Shutting down: Master
